In [2]:
#Author: Deidre Mensah
#Date:05/09/2020
#Purpose: to generate reports using local business analyst data

import arcpy, csv
import googlemaps
from arcpy import ba
from tkinter import filedialog
import requests
import requests_cache
import glob
import os

In [3]:
# Set environment settings
arcpy.env.workspace = filedialog.askdirectory()
wksp = arcpy.env.workspace

# enables overwriting the output
arcpy.env.overwriteOutput = True

# access data locally
arcpy.env.baDataSource = "LOCAL;;USA_ESRI_2019"

# imports toolbox
arcpy.AddToolbox("C:\Program Files\ArcGIS\Pro\Resources\ArcToolBox\Toolboxes\Business Analyst Tools.tbx")

<module 'ba'>

In [4]:
# gets your API with the API key
gmaps = googlemaps.Client(key='INSERT_API_KEY_HERE')

In [5]:
# create database to store cached results to
requests_cache.install_cache("MarketProfile_cache")

In [6]:
 # create csv to write lat and long to
wb = open("PointofInterest.csv", "w", newline='')
POI = csv.DictWriter(wb, fieldnames=["Long", "Lat"])
POI.writeheader()

In [7]:
# get address input
address = str(input("Enter your address: "))

Enter your address: 871 belleville dr, valley cottage, ny


In [9]:
# geocodes address
geocode_result = gmaps.geocode(address)

y = geocode_result[0]["geometry"]["location"]["lat"]
x = geocode_result[0]["geometry"]["location"]["lng"]

row = {"Long": x, "Lat": y}
POI.writerow(row)

# close file
wb.close()

# define name of csv file
table = "PointofInterest.csv"

In [25]:
# create file .gdb
arcpy.CreateFileGDB_management(wksp, "data.gdb")

# convert table data to point feature class
xylayer = arcpy.management.XYTableToPoint(table, wksp + "/data.gdb/PointofInterest", "Long".encode(),
                                          "Lat".encode())
print("point generated")

# copy feature class to shapefile layer we'll need to pass through the trade area rings tool
fc = arcpy.management.CopyFeatures(xylayer, "Points_Of_Interest.shp")

# add ID field
fc = arcpy.AddField_management(fc, "Address", "TEXT", "", "", "", "refcode", "NULLABLE", "REQUIRED")

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

point generated


In [20]:
# sets up read-write access to feature class
with arcpy.da.UpdateCursor(fc, "Address") as cursor:
    # loop through feature class rows
    for row in cursor:
        row = geocode_result[0]["formatted_address"]
        # update cursor with new address
        cursor.updateRow([row])
        #checks to make sure that you returned a google geocoded address
        print ("Your geocoded address is: " + geocode_result[0]["formatted_address"])

Your geocoded address is: 871 Belleville Dr, Valley Cottage, NY 10989, USA


In [21]:
# creates list of features to be deleted from workspace
in_features = ["Points_Of_Interest.shp", "Points_of_Interest_TA.shp"]

#name of folder to output folder to within workspace folder
folder_name = geocode_result[0]["address_components"][0]["short_name"] + "_" + geocode_result[0]["address_components"][1]["long_name"] + "_" + geocode_result[0]["address_components"][5]["short_name"]

print ("Your folder name is: " + folder_name)

Your folder name is: 871_Belleville Drive_NY


In [22]:
#ask user what type of report they want to run
report_type = input("Enter report type: mp = market profile, bs = Business summary, cp = Community Profile, dip = Demographic and Income Profile")

# specifies the name of the report that will be run based on user input. will also be used as folder name that we will zip up
if report_type == "mp" or report_type == "Mp" or report_type == "MP":
    report = "Market Profile"
elif report_type == "bs"or report_type == "BS" or report_type == "Bs":
    report ="Business Summary"
elif report_type == "cp"or report_type == "CP" or report_type == "Cp":
    report = "Community Profile"
elif report_type == "dip" or report_type == "DIP" or report_type == "dip":
    report = "Demographic and Income Profile"
else: 
    print ("Incorrect entry. Enter another report type.")
    
# ask user if they are running drive time or trade area rings
ringtype = input("Enter DT for Drive Times or TA for Trade Areas: ")
if ringtype == "TA" or ringtype == "ta" or ringtype == "Ta":
    ta_rings = int(input("Number of areas to generate: "))
    if ta_rings == 4:
        # generate trade area rings from user input
        ta = arcpy.ba.GenerateTradeAreaRings(fc, "Points_of_Interest_TA.shp",
                                             [input("Ring #1 (Miles): "),
                                              input("Ring #2 (Miles): "),
                                              input("Ring #3 (Miles): "),
                                              input("Ring #4 (Miles): ")], "MILES",
                                             "Address", "KEEP_OVERLAP")
        print ("Report exporting...")
        # defines report type and features of report
        arcpy.SummaryReports_ba(ta, report, wksp + "/" + report+ "/" + folder_name,
                                                                    "INDIVIDUAL_FEATURES", "CREATE_REPORT_PER_TEMPLATE", 
                                                                    "PDF", "Address", None, None, "STORE_LAT", "STORE_LON", "RING", "AREA_DESC",
                                                                    report + "Report", "Prepared By Business Analyst Pro")
        print ("Report Generated")
        # Execute FeatureClassToGeodatabase
        arcpy.FeatureClassToGeodatabase_conversion(in_features, wksp + "/data.gdb")
        # delete shapefiles
        arcpy.management.Delete(r"'Points_Of_Interest.shp';'Points_of_Interest_TA.shp'")

    elif ta_rings == 3:
        ta = arcpy.ba.GenerateTradeAreaRings(fc, "Points_of_Interest_TA.shp",
                                             [input("Ring #1 (Miles): "),
                                              input("Ring #2 (Miles): "),
                                              input("Ring #3 (Miles): ")], "MILES", "Address", "KEEP_OVERLAP")
        print ("Report exporting...")
        # defines report type and features of report
        arcpy.SummaryReports_ba(ta, "Market Profile", wksp + "/" + folder_name, "INDIVIDUAL_FEATURES", "CREATE_REPORT_PER_TEMPLATE",
                                "PDF", "Address", None, None, "STORE_LAT", "STORE_LON", "RING", "AREA_DESC",
                                "Market Profile Report", "Prepared By Business Analyst Pro")
        print("Report Generated")
        # Execute FeatureClassToGeodatabase
        arcpy.FeatureClassToGeodatabase_conversion(in_features, wksp + "/data.gdb")
        # delete shapefiles
        arcpy.management.Delete(r"'Points_Of_Interest.shp';'Points_of_Interest_TA.shp'")

    elif ta_rings == 2:
        ta = arcpy.ba.GenerateTradeAreaRings(fc, "Points_of_Interest_TA.shp",
                                             [input("Ring #1 (Miles): "),
                                              input("Ring #2 (Miles): ")], "MILES",
                                             "Address", "KEEP_OVERLAP")
        print ("Report exporting...")
        # defines report type and features of report
        arcpy.SummaryReports_ba(ta, "Market Profile", wksp + "/" + folder_name, "INDIVIDUAL_FEATURES", "CREATE_REPORT_PER_TEMPLATE",
                                "PDF", "Address", None, None, "STORE_LAT", "STORE_LON", "RING", "AREA_DESC",
                                "Market Profile Report", "Prepared By Business Analyst Pro")
        print("Report Generated")
        # Execute FeatureClassToGeodatabase
        arcpy.FeatureClassToGeodatabase_conversion(in_features, wksp + "/data.gdb")
        # delete shapefiles
        arcpy.management.Delete(r"'Points_Of_Interest.shp';'Points_of_Interest_TA.shp'")

    elif ta_rings == 1:
        ta = arcpy.ba.GenerateTradeAreaRings(fc, "Points_of_Interest_TA.shp",
                                             [input("Ring #1 (Miles): ")], "MILES",
                                             "Address", "KEEP_OVERLAP")
        print ("Report exporting...")
        # defines report type and features of report
        arcpy.SummaryReports_ba(ta, "Market Profile", wksp + "/" + folder_name, "INDIVIDUAL_FEATURES",
                                "CREATE_REPORT_PER_TEMPLATE",
                                "PDF", "Address", None, None, "STORE_LAT", "STORE_LON", "RING", "AREA_DESC",
                                "Market Profile Report", "Prepared By Business Analyst Pro")
        print("Report Generated")
        # Execute FeatureClassToGeodatabase
        arcpy.FeatureClassToGeodatabase_conversion(in_features, wksp + "/data.gdb")
        # delete shapefiles
        arcpy.management.Delete(r"'Points_Of_Interest.shp';'Points_of_Interest_TA.shp'")

    else:
        print("ERROR: Enter a number between 1 and 4.")

elif ringtype == "DT" or ringtype == "dt" or ringtype == "Dt":
    # ask user how many rings they want to generate
    drive_time_area = int(input("Number of rings to generate: "))
    if drive_time_area == 4:
        ta = arcpy.ba.GenerateDriveTimeTradeArea(fc, wksp + "/data.gdb/Point_of_Interest_DT",
                                                 "Driving Time", [input("Ring #1 - Drive Time (Min): "),
                                                                  input("Ring #2 - Drive Time (Min): "),
                                                                  input("Ring #3 - Drive Time (Min): "),
                                                                  input("Ring #4 - Drive Time (Min): ")], "MINUTES",
                                                 None,
                                                 "OVERLAP", "KEEP_OVERLAP",
                                                 "TOWARD_STORES", None, "TIME_ZONE_AT_LOCATION", None, "STANDARD")
        print ("Report exporting...")
        arcpy.SummaryReports_ba(ta, "Market Profile", wksp + "/" + folder_name, "INDIVIDUAL_FEATURES",
                                "CREATE_REPORT_PER_TEMPLATE",
                                "PDF", "Address", None, None, "STORE_LAT", "STORE_LON", "RING", "AREA_DESC",
                                "Market Profile Report", "Prepared By Business Analyst Pro")
        print("report generated")
        # Execute FeatureClassToGeodatabase
        arcpy.FeatureClassToGeodatabase_conversion("Points_Of_Interest.shp", wksp + "/data.gdb")
        # delete shapefiles
        arcpy.management.Delete(r"'Points_Of_Interest.shp'")

    elif drive_time_area == 3:
        ta = arcpy.ba.GenerateDriveTimeTradeArea(fc, wksp + "/data.gdb/Point_of_Interest_DT",
                                                 "Driving Time", [input("Ring #1 - Drive Time (Min): "),
                                                                  input("Ring #2 - Drive Time (Min): "),
                                                                  input("Ring #3 - Drive Time (Min): ")], "MINUTES", None, "OVERLAP", "KEEP_OVERLAP", "TOWARD_STORES", None, "TIME_ZONE_AT_LOCATION", None, "STANDARD")
        print ("Report exporting...")
        arcpy.SummaryReports_ba(ta, "Market Profile", wksp + "/" + folder_name, "INDIVIDUAL_FEATURES",
                                "CREATE_REPORT_PER_TEMPLATE",
                                "PDF", "Address", None, None, "STORE_LAT", "STORE_LON", "RING", "AREA_DESC",
                                "Market Profile Report", "Prepared By Business Analyst Pro")
        print("Report Generated")
        # Execute FeatureClassToGeodatabase
        arcpy.FeatureClassToGeodatabase_conversion("Points_Of_Interest.shp", wksp + "/data.gdb")
        # delete shapefiles
        arcpy.management.Delete(r"'Points_Of_Interest.shp'")

    elif drive_time_area == 2:
        ta = arcpy.ba.GenerateDriveTimeTradeArea(fc, wksp + "/data.gdb/Point_of_Interest_DT",
                                                 "Driving Time", [input("Ring #1 - Drive Time (Min): "),
                                                                  input("Ring #2 - Drive Time (Min): ")], "MINUTES",
                                                 None,
                                                 "OVERLAP", "KEEP_OVERLAP",
                                                 "TOWARD_STORES", None, "TIME_ZONE_AT_LOCATION", None, "STANDARD")
        print ("Report exporting...")
        arcpy.SummaryReports_ba(ta, "Market Profile", wksp + "/" + folder_name, "INDIVIDUAL_FEATURES",
                                "CREATE_REPORT_PER_TEMPLATE",
                                "PDF", "Address", None, None, "STORE_LAT", "STORE_LON", "RING", "AREA_DESC",
                                "Market Profile Report", "Prepared By Business Analyst Pro")
        print("Report Generated")
        # Execute FeatureClassToGeodatabase
        arcpy.FeatureClassToGeodatabase_conversion("Points_Of_Interest.shp", wksp + "/data.gdb")
        # delete shapefiles
        arcpy.management.Delete(r"'Points_Of_Interest.shp'")

    elif drive_time_area == 1:
        ta = arcpy.ba.GenerateDriveTimeTradeArea(fc, wksp + "/data.gdb/Point_of_Interest_DT",
                                                 "Driving Time", [input("Ring #1 - Drive Time (Min): ")], "MINUTES",
                                                 None,
                                                 "OVERLAP", "KEEP_OVERLAP",
                                                 "TOWARD_STORES", None, "TIME_ZONE_AT_LOCATION", None, "STANDARD")
        print ("Report exporting...")
        arcpy.SummaryReports_ba(ta, "Market Profile", wksp + "/" + folder_name, "INDIVIDUAL_FEATURES",
                                "CREATE_REPORT_PER_TEMPLATE",
                                "PDF", "Address", None, None, "STORE_LAT", "STORE_LON", "RING", "AREA_DESC",
                                "Market Profile Report", "Prepared By Business Analyst Pro")
        print("Report Generated")
        # Execute FeatureClassToGeodatabase
        arcpy.FeatureClassToGeodatabase_conversion("Points_Of_Interest.shp", wksp + "/data.gdb")
        # delete shapefiles
        arcpy.management.Delete(r"'Points_Of_Interest.shp'")

    else:
        print("ERROR: Enter a number between 1 and 4.")
else:
    print("ERROR: Enter in DT or TA.")

Enter report type: mp = market profile, bs = Business summary, cp = Community Profile, dip = Demographic and Income Profilemp
Enter DT for Drive Times or TA for Trade Areas: ta
Number of areas to generate: 2
Ring #1 (Miles): 1
Ring #2 (Miles): 2
Report exporting...
Report Generated
